In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

CHROME_DRIVER_PATH = "C:\\Users\\82108\\.wdm\\drivers\\chromedriver\\win64\\127.0.6533.72\\chromedriver-win32\\chromedriver.exe"
service = Service(executable_path=CHROME_DRIVER_PATH)
options = Options()
driver = webdriver.Chrome(service=service, options=options)

name_list = []
brand_list = []
price_list = [] 
sale_price_list = [] 
volume_list = []
skin_type_list = []
ingredient_list = []
review_list = []
all_review_list = []

def make_df(name, brand, price, sale_price, volume, skin_type, ingredient, indiv_product_reviews):
    name_list.append(name)
    brand_list.append(brand)
    price_list.append(price)
    sale_price_list.append(sale_price)
    volume_list.append(volume)
    skin_type_list.append(skin_type)
    ingredient_list.append(ingredient)
    review_list.append(indiv_product_reviews)

# def scroll_to_end(driver):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
#     )

def review_next_page(review_page_num):
    try:
        page_bar = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')
        for button in page_bar:
            if button.get_attribute("data-page-no") == str(review_page_num):
                driver.execute_script("arguments[0].click();", button)
                # WebDriverWait(driver, 10).until(
                #     EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasList')))
                
                break
    except NoSuchElementException:
        pass
    

def extract_review(driver):
    # 모든 페이지 버튼 요소를 찾기
    page_buttons = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')




    indiv_product_reviews = []
    for review_page_num in range(1,11): 
        try:
            review_next_page(review_page_num)
            WebDriverWait(driver, 60).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#gdasList > li > div.review_cont > div.txt_inner'))
            )
            for review_num in range(1, 11):
                try:
                    review = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({review_num}) > div.review_cont > div.txt_inner').text
                    indiv_product_reviews.append(review)
                    all_review_list.append(review)
                except (NoSuchElementException, StaleElementReferenceException):
                    pass
        except (NoSuchElementException, StaleElementReferenceException):
            break

    return indiv_product_reviews


def extract_data(driver, para, product_number):
    try:
        product_button = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({para}) > li:nth-child({product_number}) > div > div > a > p') 
        product_button.click()
        name = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name'))
        ).text
        print(name)

        brand = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#moveBrandShop'))
        ).text
        print(brand)

        try:
            sale_price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong')  
            sale_price = sale_price_element.text
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike') 
            price = price_element.text
        except NoSuchElementException:
            sale_price = None
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong') 
            price = price_element.text

        print(price)
        print(sale_price)
        
        info_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#buyInfo > a')))
        info_button.click()
        
        volume = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd'))).text
        print(volume)
        
        skin_type = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd'))).text
        print(skin_type)
        
        ingredient = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd'))).text
        print(ingredient)
        
        info_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#reviewInfo > a')))
        info_button.click()
        
        indiv_product_reviews = extract_review(driver)
        
        make_df(name, brand, price, sale_price, volume, skin_type, ingredient, indiv_product_reviews)
        driver.back()
        
    except NoSuchElementException:
        driver.back()


try:
    stop_flag = False
    for page in range(8, 9):
        if stop_flag:
            break
        for para in range(9, 10):
            if stop_flag:
                break
            for product_number in range(1, 2):
                if stop_flag:
                    break
                url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat100000100010013_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
                driver.get(url)

                try:
                    extract_data(driver, para, product_number)
                except NoSuchElementException:
                    stop_flag = True
                    break
    
    # 데이터프레임 생성 및 출력
    info_data = {'name': name_list, 'brand': brand_list, 'price': price_list, 'sale_price': sale_price_list, 'volume': volume_list, 'skin_type': skin_type_list, 'ingredient': ingredient_list, 'review' : review_list}
    df_info = pd.DataFrame(info_data)
    all_review = {'review':all_review_list}
    df_all_review = pd.DataFrame(all_review)
finally:
    # 브라우저 종료
    driver.quit()     



숨37 마이크로액티브 토닝워터 150ml (+화장솜50매)
숨37
50,000
None
[본품] 150ml [증정] 화장솜 50매
모든피부용
효모발효용해여과물, 프로판다이올, 부틸렌글라이콜, 변성알코올, 정제수, 트레할로오스, 1,2-헥산다이올, 다이프로필렌글라이콜, 글리세린, C13-15알케인, 스쿠알란, 판테놀, 글루코노락톤, 하이드록시에틸피페라진에탄설포닉애씨드, 아이소노닐아이소노나노에이트, 솔비톨, 트로메타민, 비니거, 아데노신, 효모폴리펩타이드, 낫토검, 토코페롤, 에틸헥실글리세린, 베르가모트오일, 해바라기씨오일, 일랑일랑꽃오일, 다마스크장미꽃오일, 트라이소듐이디티에이, 리모넨, 리날룰, 벤질벤조에이트, 시트로넬올, 벤질살리실레이트, 제라니올, 파네솔


In [ ]:
df_info

name brand   price sale_price  \
0  숨37 마이크로액티브 토닝워터 150ml (+화장솜50매)   숨37  50,000       None   

                    volume skin_type  \
0  [본품] 150ml [증정] 화장솜 50매     모든피부용   

                                                                                                                                                                                                                                                                                              ingredient  \
0  효모발효용해여과물, 프로판다이올, 부틸렌글라이콜, 변성알코올, 정제수, 트레할로오스, 1,2-헥산다이올, 다이프로필렌글라이콜, 글리세린, C13-15알케인, 스쿠알란, 판테놀, 글루코노락톤, 하이드록시에틸피페라진에탄설포닉애씨드, 아이소노닐아이소노나노에이트, 솔비톨, 트로메타민, 비니거, 아데노신, 효모폴리펩타이드, 낫토검, 토코페롤, 에틸헥실글리세린, 베르가모트오일, 해바라기씨오일, 일랑일랑꽃오일, 다마스크장미꽃오일, 트라이소듐이디티에이, 리모넨, 리날룰, 벤질벤조에이트, 시트로넬올, 벤질살리실레이트, 제라니올, 파네솔   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
df_all_review

,review
0,"벌써 두병째 공병보는 숨의 토너 제품이에요! 나오자마자 구매해서 지금까지 꾸준하게 사용하고있는데 심플 이 베스트,, 무난한 사용감으로 없어지면 속상한 그런 제품으로 자리매김한 제품이랍니다..🫶\n\n촉촉하면서도 기름지지 않고, 닦토 흡토 둘다 사용할 수 있는 정말 찐 무난템이라 매일 손이 가는 재품인 것 같아요\n\n흡토로 사용시에도 흡수가 빨리 되는 편이라서 아침에 톡톡 얹어주기에도 좋았고 수분감을 쫙 채워주는 느낌이라 건성인 제가 사용하기에도 무리가 없었던 제품이라 특히나 좋았답니다!\n\n올리브영에서도 숨을 만나볼 수 있다니,,, 너무 좋아요ㅠㅠ 제 최애 브랜드인만큼 기대하고 믿고 사용했었고 기대 이상으로 좋았던 제품이라 앞으로도 계속 구매해서 사용할 것 같아요!\n\n숨의 스킨케어 좋아하셨던 분들이나 토너 필요하셨던 분들이라면 추천드려요!"
1,*한달사용후기입니다🙂\n\n유리병으로 되어있어서 고급져요.\n흔들어 사용해야하는게 좀 불편하고 수부지인 전 코나 이마 볼쪽에 뾰루지가 올라오더라구요 ㅠㅠ\n근데 극건정인 저희 어머니는 아주 만족하면서 사용하고 계셔서 별4개 줬여요.
2,체험단으로 받아서 사용해보고 너무 좋아서 내돈내산했어요!\n백화점 입점 브랜드치고 가격이 너무 비싸지도 않고 용량도 많아요.\n민감한 제 피부에도 잘 맞았고 화장솜 세트구성에 할인+쿠폰 적용해서 인터넷 최저가보다 싸게 구매했어요:)\n역시 올영..\n화장솜은 여행갔을 때 휴대용으로 갖고가기도 좋을 것 같고 스킨용기 딱 봤을 때 엄청 고급져보여요.\n흔들어서 사용하세요!
3,"2.5%의 오일층과 97.5%의 워터층으로 나누어져있어서 흔들어서 사용해야하는 토닝워터입니다. 화장솜이 내장되어있어서 같이 사용하기 좋아요. 저자극 발효 PHA함유로 데일리 각질케어와 피부진정 효과를 볼 수 있어요. 1회 사용만으로 묵은 각질 감소, 2.5배 투명해지는 피부를 느낄수있고, 저는 수분부족형지성인데 여름에 사용해도 무겁지 않아서 좋아요."
4,기존에도 숨의 제품들을 너무 잘 사용해서 이번에 새롭게 출시된 토너도 꼭 사용해보고싶었어요\n\n내돈내산+올영체험단까지 총 두병을 사용하고있는 제품인만큼 더 정확하게 알려드릴 수 있을 것 같아요🎀\n\n기름층과 토너층이 나뉘어져있는데 기름층의 비율이 크지 않아 기름진느낌보다는 가벼운 제품이라는 생각이 들었어요:)\n\n완전 건성용보다는 건성부터 복합성까지 두루 챙길 수 있는 토너라는 생각이 들었답니다\n\n흡수가 빠르고 촉촉해서 자주 손이 가는 제품인데요 요즘같이 더운 날씨에 저같은 건성은 수분감 충전은 필요하지만 유분감이 너무 많으면 불편할 수 있는데 이건 흡수도 빠르고 촉촉함도 채워지면서 피부를 리프레쉬 해주는 느낌이라 사용하기 좋았어요\n\n용량도 넉넉하고 프리미엄 브랜드 치고는 가격대도 저렴해서 사용하기 좋은 것 같아요\n\n닦토로도 흡토로도 사용하기 좋았어서 무난한 토너 찾으셨다면 추천! 만족하실 제품이에요
...,...
77,지복합성 약간의 속건조 있는데 도움 되요\n20대에서 30대 초반까진 무난하게 사용할 수 있을듯 한데 30대 중반부턴 좀더 보습과 오일리함이 더 필요한것같아요
78,시트러스향\n오일과 분리되어 흔들어 써야해요\n가벼우나 보습감도 갖춘\n그러나 건성에게는 추천하지 않아요
79,화장 지우는데 깨끗하고 깔끔하게 세정이 잘 됩니다. 재구매 의사 있음.
80,이 제품을 사용하면 피부가 좋아지는 느낌을 받습니다. 아주 좋아요
